# NFW-profile fitting

In [31]:
import Clusters as c
import numpy as np
import readsnap
import astropy
from matplotlib import pyplot as plt, rc
from astropy.cosmology import FlatLambdaCDM
from astropy import cosmology
from astropy import units as u

cosmo = FlatLambdaCDM(H0=67.74, Om0=0.3089)

rc('figure', figsize=(8,6))
rc('font', size=18)
rc('lines', linewidth=3)
rc('axes', linewidth=2)
rc('xtick.major', width=2)
rc('ytick.major', width=2)

def in_files(files):
    data = open(files, 'r')
    lines = data.readlines()
    id_halo = []
    z_halo = []
    Mvir = []
    for k in range(len(lines))[1:]:
        [ids, dd, z, mv, dd, dd, dd, dd, dd, dd, dd, dd, dd, dd] = lines[k].split()
        id_halo.append(int(ids))
        z_halo.append(float(z))
        Mvir.append(float(mv))  # [Msun/h]
    id_halo = np.asarray(id_halo)
    z_halo = np.asarray(z_halo)
    Mvir = np.asarray(Mvir)
    return id_halo, z_halo, Mvir

def out_files(files):
    data = open(files, 'r')
    lines = data.readlines()
    id_lens = []
    theta_E = []
    Mlens = []
    redshift = []
    con = []
    vrms = []
    subpos = []
    for k in range(len(lines)):
        [ids, theta, mv, redsh, c, v, dd, x, y, z] = lines[k].split()
        id_lens.append(int(ids))
        theta_E.append(float(theta))
        Mlens.append(float(mv))  # [Msun/h]
        redshift.append(float(redsh))
        con.append(float(c))
        vrms.append(float(v))
        subpos.append([float(x), float(y), float(z)])  # [Mpc]
    id_lens = np.asarray(id_lens)
    theta_E = np.asarray(theta_E)
    Mlens = np.asarray(Mlens)
    redshift = np.asarray(redshift)
    con = np.asarray(con)
    vrms = np.asarray(vrms)
    subpos = np.asarray(subpos)
    return id_lens, theta_E, Mlens, redshift, subpos


def find_nearest(array, value):
    idx = np.argmin(np.abs(array-value))
    return idx


def check_in_sphere(centre, pos, Rth):
    r = np.sqrt((centre[0]-pos[:, 0])**2 +
                (centre[1]-pos[:, 1])**2 +
                (centre[2]-pos[:, 2])**2)
    rmin = np.min(r)
    ids = np.where(r < Rth)
    return ids, rmin

In [32]:
# Load halo lensing properties

infile = '/cosma5/data/dp004/dc-beck3/lightcone/non_radiative_hydro/lenses_L62_N512_GR_pure_rerun.txt'
id_halo, z_halo, Mvir = in_files(infile)

outfile = '/cosma5/data/dp004/dc-beck3/lightcone/non_radiative_hydro/outlens_L62_N512_GR_pure_rerun.dat'
id_lens, theta_E, Mlens, redshift, subpos = out_files(outfile)


# Find indices of results belonging to one lens
idlens = np.unique(id_lens)
lens_indx = []
Mtrue = []
for i in idlens:
    lens_indx.append(np.where(id_lens == i))
    Mtrue.append(Mvir[np.where(id_halo == i)])
Mtrue = np.asarray(Mtrue)

# Find max Einstein ring for each lens
Mlensmax = np.zeros(len(id_lens))
theta_Emax = np.zeros(len(lens_indx))
redshiftmax = np.zeros(len(lens_indx))
subposmax = np.zeros((len(lens_indx), 3))
for i in range(len(idlens)):
    try:
        indx = np.argmax(theta_E[lens_indx[i][0]])
        Mlensmax[i] = Mlens[lens_indx[i][0][indx]]
        theta_Emax[i] = theta_E[lens_indx[i][0][indx]]
        redshiftmax[i] = redshift[lens_indx[i][0][indx]]
        subposmax[i] = subpos[lens_indx[i][0][indx], :]
    except:
        continue
        
indx = np.where(theta_Emax != 0.0)[0]
Mlensmax = Mlensmax[indx]
Mtrue = Mtrue[indx]
theta_Emax = theta_Emax[indx]
redshiftmax = redshiftmax[indx]
subposmax = subposmax[indx]

# Einstein Radius: convert argsec to kpc
R_E = [theta_Emax[i]*(1/60)*cosmo.kpc_proper_per_arcmin(redshiftmax[i])*(u.arcmin/u.kpc) for i in range(len(theta_Emax))]
R_E = np.asarray(R_E)

In [34]:
# Load particles

sim_dir = '/cosma6/data/dp004/dc-arno1/SZ_project/non_radiative_hydro/L62_N512_GR_pure_rerun/'
snapfile = sim_dir+'/snapdir_%03d/snap_%03d.0'

# Find snapshot numbers
snap_tot_num = len(open(sim_dir+'arepo/output_list_new.txt', 'r').readlines())-1
snapnums = np.arange(snap_tot_num+1)
# Find snapshot redshifts
z_sim = []
for i in range(snap_tot_num, -1, -1):
    header = readsnap.snapshot_header(snapfile % (i, i))
    z_sim.append(header.redshift)

snap_tot_num = len(open(sim_dir+'arepo/output_list_new.txt', 'r').readlines())-1
header = readsnap.snapshot_header(snapfile % (snap_tot_num, snap_tot_num))
cosmosim = {'omega_M_0' : header.omega_m,
            'omega_lambda_0' : header.omega_l,
            'omega_k_0' : 0.0,
            'h' : header.hubble}

R_E *=1e-3  # convert to Mpc
sim_dir = '/cosma6/data/dp004/dc-arno1/SZ_project/non_radiative_hydro/L62_N512_GR_pure_rerun/'
snapfile = sim_dir+'snapdir_%03d/snap_%03d'
ratio = np.zeros(len(R_E))
for l in range(len(R_E)):
    snapnum = find_nearest(z_sim, redshiftmax[l])
    snap = snapfile % (snapnum, snapnum)
    dmpos = readsnap.read_block(snap, 'POS ', parttype=0)
    dmmass = readsnap.read_block(snap, 'MASS', parttype=0)
    centre = subposmax[l]
    xlims = [3e0, 2e3]
    print('Start fitting')
    (sim_r, sim_measure, dum, dum, ndum, ndum, dum) = c.densprof(0, xlims[0], xlims[1], snapfile=snap, xc=centre[0], yc=centre[1], zc=centre[2])
    
    plt.loglog()
    plt.xlim(xlims)
    plt.scatter(sim_r, sim_measure, c='b')
    plt.xlabel('R [Mpc]')
    plt.ylabel(r'$\rho$ [M_{\odot}/Mpc^{3}]')
    plt.show()
    break

Start fitting
FoF group:  0
Nclasses:  100
masss 61.9999998362
Gas density profile
logarithmic profile
---
rmin, rmax: 0.47712125472 3.30102999566  center: 25.68249 30.37167 51.10726 number of parts: 129295819
---
12929581  /  129295819
25859162  /  129295819
38788743  /  129295819
51718324  /  129295819
64647905  /  129295819
77577486  /  129295819
90507067  /  129295819
103436648  /  129295819
116366229  /  129295819
129295810  /  129295819


NameError: name 'iplot' is not defined

In [ ]:
2